Counting bigrams leading up to QUERY and after QUERY

In [3]:
import json

query = 'probably'

#load json file of probablies
with open('app/json_files/combined/probably.json') as json_data:
	parsed_json = json.load(json_data)

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

#define stopwords
punctuation = list(string.punctuation)
digits = list(string.digits)
extra = [query, '\'s', '\'\'', '``', '--', '...', 'n\'t', '\'d', '\'ll', '\'re', '\'m']
stop = stopwords.words('english') + punctuation + digits + extra

Start by looking at the words that comes after "probably" in each sentence, taking out NLTK's English stopwords.

In [17]:
#def nextwords

def find_nextwords(parsed_json, stop):
    next_words = []
    missing_query_list = []
    for item in parsed_json:
        title = word_tokenize(item['title'].lower().strip(']').strip('['))
        try:
            query_index = title.index(query)
        except ValueError:
            missing_query_list.append(item['title'])
            continue
        try:
            next_word = title[query_index+1]
        except IndexError:
            next_word = 'last_word'
            continue
        if next_word not in stop:
            next_words.append(next_word)
    return next_words

next_words = find_nextwords(parsed_json,stop)

In [9]:
#count nextwords
from collections import Counter

def count_common(word_list):
    count_all= Counter()
    count_all.update(word_list)
    print(len(word_list))
    print(count_all.most_common(25))

count_common(next_words)

543
[('taken', 30), ('congressional', 16), ('near', 12), ('used', 12), ('ca', 11), ('march', 10), ('end', 9), ('original', 7), ('orville', 6), ('new', 6), ('salem', 6), ('never', 5), ('part', 5), ('detroit', 5), ('michigan', 5), ('1920', 5), ('early', 4), ('going', 4), ('hungarian', 4), ('c.', 4), ('photographed', 4), ('installed', 4), ('south', 4), ('added', 4), ('around', 4)]


However, it turns out stopwords like "in" and "at" are kind of interesting here. So let's look at the same set of titles with a different set of stopwords, this time including prepositions.

In [10]:
prepositions = ['in', 'at', 'on']
new_stopwords = [ w for w in stop if w not in prepositions]
no_stopwords = extra + punctuation + digits + extra

In [11]:
next_with_prepositions, missing_query_list = find_nextwords(parsed_json, new_stopwords)
next_no_stop, missing_query_list = find_nextwords(parsed_json, no_stopwords)

print(count_common(next_with_prepositions))
print(count_common(next_no_stop))

790
[('in', 179), ('at', 52), ('taken', 30), ('on', 16), ('congressional', 16), ('near', 12), ('used', 12), ('ca', 11), ('march', 10), ('end', 9), ('original', 7), ('orville', 6), ('new', 6), ('salem', 6), ('never', 5), ('part', 5), ('detroit', 5), ('michigan', 5), ('1920', 5), ('early', 4), ('going', 4), ('hungarian', 4), ('c.', 4), ('photographed', 4), ('installed', 4)]
None
1106
[('in', 179), ('the', 61), ('a', 54), ('at', 52), ('taken', 30), ('from', 28), ('for', 22), ('during', 20), ('be', 17), ('on', 16), ('congressional', 16), ('of', 14), ('not', 14), ('under', 13), ('near', 12), ('used', 12), ('ca', 11), ('march', 10), ('end', 9), ('original', 7), ('orville', 6), ('to', 6), ('new', 6), ('about', 6), ('his', 6)]
None


Next let's look at what else happens after the query, by creating bigrams (or trigrams?) from each title after the word probably

In [35]:
def find_phrases(data, query):
    next_phrases = {}
    missing_query_list = []
    last_phrases = {}
    
    for item in data:
        title = word_tokenize(item['title'].lower().strip(']').strip('[').strip('"'))
        try:
            query_index = title.index(query)
        
        except ValueError:
            missing_query_list.append(item['title'])
            continue
            
        if query_index == len(title)-1:
            last_phrase = title[:query_index]
            last_phrases[item['pk']] = last_phrase
        else:
            if title[query_index-1] == '(':
                next_phrase = title[(query_index-1):]
            else:
                next_phrase = title[query_index:]
            next_phrases[item['pk']] = next_phrase
    
    return next_phrases, missing_query_list, last_phrases

next_phrases, missing_query_list, last_phrases = find_phrases(parsed_json, 'probably')

print(next_phrases)

{'00649511': ['probably', 'at', 'the', 'wedding', 'of', 'jack', 'and', 'jacqueline', 'kennedy'], '00650393': ['probably', 'was', 'good', 'for', 'the', 'lad', '!', '/'], '00651112': ['probably', 'dr.', 'steve', 'sheppard', ',', 'brother', 'of', 'dr.', 'sam', 'sheppard'], '00651827': ['probably', 'boy', ',', 'close', 'up', 'of', 'face'], '00652703': ['probably', 'from', 'a', 'child', ',', 'following', 'mr.', 'ruby', "'s", 'assassination', 'of', 'lee', 'harvey', 'oswald'], '2001696171': ['probably', 'loaded', 'with', 'missiles', 'under', 'the', 'canvas', 'cover', 'seen', 'on', 'deck'], '2001696569': ['probably', 'wilbur', 'wright', 'and', 'charles', 'e.', 'taylor', '.', 'the', 'catapult', 'launching', 'device', 'appears', 'for', 'the', 'first', 'time', 'in', 'a', 'photograph'], '2001696576': ['probably', 'wilbur', 'and', 'orville', 'standing', 'on', 'either', 'side', ';', 'huffman', 'prairie', ',', 'dayton', ',', 'ohio'], '2001696641': ['probably', 'orville', 'acting', 'as', 'instructor',

Let's look at what's before the probably. It's really only interesting if it's the last word or in parathenses.

In [13]:
last_phrases = []
for item in parsed_json:
        title = word_tokenize(item['title'].lower())
        try:
            query_index = title.index(query)
            if query_index >= len(title)-1:
                last_phrase = title[:query_index+1]
                last_phrases.append(title)
        except ValueError:
            continue

print (last_phrases)

[['minnesota', '--', 'probably'], ['sen.', 'ford', '/', 'rep.', 'charlie', 'probably']]


So then we can sort based on the first word of each phrase

In [42]:
    print(next_with_prepositions)

['at', 'dr.', 'boy', 'loaded', 'wilbur', 'wilbur', 'orville', 'orville', 'orville', 'orville', 'orville', 'in', 'in', 'in', 'in', 'interior', 'in', 'in', 'at', 'in', 'in', 'in', 'in', 'in', 'lehigh', 'chartered', 'designed', 'in', 'at', 'intended', 'in', 'waacs', 'waacs', 'kitaiskaia', 'orville', 'in', 'in', 'in', 'in', 'in', 'taken', 'medieval', 'in', 'thomas', 'mary', 'taken', 'george', 'in', 'granddaughter', 'near', 'near', 'used', 'in', 'at', 'new', 'new', 'beatrice', 'copied', 'taken', 'used', 'made', 'in', 'in', 'in', 'russian', 'american', 'at', 'at', 'at', 'at', 'at', 'at', 'at', 'in', 'in', 'novorossiik', 'journalists', 'journalists', 'sailors', 'at', 'near', 'in', 'journalists', 'commuters', 'in', 'in', 'german', 'german-american', 'belonging', 'made', 'intended', 'in', 'intended', 'in', 'book', 'illustrations', 'in', 'near', 'used', 'in', 'celebrating', 'in', 'in', 'representing', 'in', 'in', 'drawings', 'on', 'in', 'in', 'on', 'jap', 'in', 'in', 'les', 'somewhere', 'appear'

In [50]:
categories = {}

for item in next_with_prepositions:
    if item in categories:
        pass
    else:
        categories[item] = []

for item in next_phrases:
    test = item[1]
    if test in categories:
        title_list = categories[test] 
        title_list.append("".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in item]).strip())

print(categories)
    

{'at': ['probably at the wedding of jack and jacqueline kennedy', 'probably at smithsonian institution', 'probably at her home in auburn, new york', 'probably at chicago stock yards', 'probably at the time of the portsmouth peace conference', 'probably at the time of the portsmouth peace conference', 'probably at the time of the portsmouth peace conference', 'probably at the time of the portsmouth peace conference', 'probably at the time of the portsmouth peace conference', 'probably at the time of the portsmouth peace conference', 'probably at the time of the portsmouth peace conference', 'probably at the boise golf club', 'probably at a recording site in belle glade, florida, 1935', 'probably at the parthenon].', 'probably at a recording site, belle glade, florida', 'probably at fort custer, michigan', 'probably at a meeting of the senate committee on foreign relations', 'probably at their home', 'probably at their home', 'probably at camp griffin, langley, virginia', 'probably at ca

In [43]:
def make_dict(data, query):
    next_words = find_nextwords(data, query)
    next_phrases, missing_query_list, last_phrases = find_phrases(data, 'probably')

    categories = {}

    for next_word in next_words:
        categories[next_word] = {}
    
    details= {}
    for item in data:
        title=item['title']
        pk =item['pk']
        details[pk] ={
            'pk': pk,
            'url': item['links']['item'],
            'img':item['image']['full'],
        }
        
        for item in next_phrases:
            clean_title = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in next_phrases[item]])
            if item == pk:
                details[pk]['title'] = clean_title
    return details

make_dict(parsed_json, 'probably')

{'00649511': {'pk': '00649511',
  'url': '//www.loc.gov/pictures/item/00649511/',
  'img': '//www.loc.gov/pictures/cdn/service/pnp/cph/3c20000/3c22000/3c22000/3c22083_150px.jpg',
  'title': ' probably at the wedding of jack and jacqueline kennedy'},
 '00650393': {'pk': '00650393',
  'url': '//www.loc.gov/pictures/item/00650393/',
  'img': '//www.loc.gov/pictures/cdn/service/pnp/ppmsc/02800/02828r.jpg',
  'title': ' probably was good for the lad!/'},
 '00651112': {'pk': '00651112',
  'url': '//www.loc.gov/pictures/item/00651112/',
  'img': '//www.loc.gov/pictures/cdn/service/pnp/cph/3c20000/3c26000/3c26200/3c26277_150px.jpg',
  'title': ' probably dr. steve sheppard, brother of dr. sam sheppard'},
 '00651827': {'pk': '00651827',
  'url': '//www.loc.gov/pictures/item/00651827/',
  'img': '//www.loc.gov/pictures/cdn/service/pnp/ds/00100/00186r.jpg',
  'title': ' probably boy, close up of face'},
 '00652703': {'pk': '00652703',
  'url': '//www.loc.gov/pictures/item/00652703/',
  'img': '//